In [1]:
from mmdet.apis import init_detector, inference_detector
import mmcv
import matplotlib.pyplot as plt
import cv2
import os
import glob
import json
from tqdm.notebook import tqdm
import numpy as np
import logging
import math
from tracking import Tracker

In [2]:
### Path to OCR of the video
data = open('../../data/ocr_results/results/ocr_with_gameclockrunning/2018-11-28_Virginia_at_Maryland/2018-11-28_Virginia_at_Maryland_ocr.json')
data = json.load(data)

### Config and model weights path
config_file = 'configs/yolo/custom_yolov3_d53_mstrain-608_273e_coco.py'
checkpoint_file = 'work_dirs/yolov3_d53_mstrain-608_273e_coco/epoch_273.pth'

### Path to the video
video_path = '../../data/videos/videos/2018-11-28_Virginia_at_Maryland.mp4'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

In [ ]:
### Single Video inference

logging.basicConfig(level=logging.DEBUG)
### Comment if you want to DEBUG
logging.disable(logging.DEBUG)

### Load the video using MMCV
video = mmcv.VideoReader(video_path)

### Temp variables
en = 0
proc = 0
pbar = tqdm(total=video.frame_cnt)
has_pred = 0


### Create the tracker
tracker = Tracker()


### Start prediction
for frame in video:
    
    ### Check if it is game moment using OCR
    idx = str(en)
    if not data['results'][idx]['score_bug_present'] or not data['results'][idx]['game_clock_running']:
        en += 1
        continue
    
    
    ### Predict the image and update stats
    result = inference_detector(model, frame)
    if len(result[0]) != 0: has_pred+=1
        
    ### Update tracker results
    tracker.update(result)
    
    ### Update stats
    en += 1
    proc += 1
    if proc > 1000: break  
    pbar.update(1)
    
    
pbar.close()

In [ ]:
### Here I run the function which search for missing detections intervals in tracker.current_tracks 
### and interpolates its values. The below threshold is pointing how many missed values to interpolate.
how_much_to_interpoate = 6
tracker.calc_missing_intervals(how_much_to_interpoate)

In [14]:
### DEMO FOR MULTIPLE VIDEOS

logging.basicConfig(level=logging.DEBUG)
### Comment if you want to DEBUG
logging.disable(logging.DEBUG)

all_videos = glob.glob('./demo_vids/*.mp4')

track_dict = {}


all_tracks = []
for video_path in all_videos:
    ### Temp variables
    en = 0
    proc = 0
    has_pred = 0

    tracker = Tracker()
    video = mmcv.VideoReader(video_path)
    pbar = tqdm(total=video.frame_cnt)
    ### Start prediction
    for frame in video:


        ### Predict the image and update stats
        result = inference_detector(model, frame)
        if len(result[0]) != 0: has_pred+=1

        ### Update tracker results
        tracker.update(result)

        ### Update stats
        en += 1
        proc += 1
#         if proc > 1000: break  
        pbar.update(1)


    pbar.close()
    how_much_to_interpoate = 6
    tracker.calc_missing_intervals(how_much_to_interpoate)
    track_dict[os.path.basename(video_path)] = tracker.current_tracks
    all_tracks.extend(tracker.current_tracks)

In [16]:
final_track = {}
for k, v in track_dict.items():
    final_track[k] = [x.tolist() if x is not None else x for x in v]

with open('./demo_vids/output/frames_res.json', 'w') as f:
    json.dump(final_track, f)

In [20]:
vid = mmcv.VideoReader('./demo_vids/1.mp4')

len(final_track['1.mp4']), vid.frame_cnt

(1631, 1631)

## Visualize tracker results

In [19]:
# test a video and show the results using BBOX
video = mmcv.VideoReader('../../data/videos/videos/2018-11-28_Virginia_at_Maryland.mp4')
out = cv2.VideoWriter('output_tracker_demo.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 15, (1280,720))

l_thickness = 2
en = 0
proc = 0
not_none = 0
for frame in video:
    idx = str(en)
    if not data['results'][idx]['score_bug_present'] or not data['results'][idx]['game_clock_running']:
        en += 1
        continue
    
    if tracker.current_tracks[proc] is not None:
        not_none += 1 
        result = np.expand_dims(tracker.current_tracks[proc][:4], axis=0)
        frame = mmcv.imshow_bboxes(frame, result, show=False, thickness=l_thickness, colors=['green'])
        
        if tracker.current_tracks[proc-1] is not None:
            curr = tracker.current_tracks[proc].astype(np.int32)
            prev = tracker.current_tracks[proc-1].astype(np.int32)
            
            #upper left
            cv2.line(frame, (curr[0], curr[1]), (prev[0], prev[1]), (0, 255, 0), thickness=l_thickness)
            #upper right 
            cv2.line(frame, (curr[2], curr[1]), (prev[2], prev[1]), (0, 255, 0), thickness=l_thickness)
            #lower left
            cv2.line(frame, (curr[0], curr[3]), (prev[0], prev[3]), (0, 255, 0), thickness=l_thickness)
            #lower right
            cv2.line(frame, (curr[2], curr[3]), (prev[2], prev[3]), (0, 255, 0), thickness=l_thickness)
            
          
    
    out.write(frame)

    en += 1
    proc += 1
    if proc > 500:break
    
    
    
out.release()

In [49]:
# test a video and show the results using CIRCLE
video = mmcv.VideoReader('../../data/videos/videos/2018-11-28_Virginia_at_Maryland.mp4')
out = cv2.VideoWriter('output_tracker_demo.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 25, (1280,720))

l_thickness = 2
en = 0
proc = 0
not_none = 0

dists = []
for frame in video:
    idx = str(en)
    if not data['results'][idx]['score_bug_present'] or not data['results'][idx]['game_clock_running']:
        en += 1
        continue
    
    if tracker.current_tracks[proc] is not None:
        not_none += 1 
#         result = np.expand_dims(tracker.current_tracks[proc][:4], axis=0)
        result = tracker.current_tracks[proc][:4]
#         print(result)
        center = (int((result[0] + result[2]) / 2), int((result[1] + result[3]) / 2))
        radius = int((result[2] - result[0]) / 2)
        cv2.circle(frame, center, radius, (0, 255, 0), thickness=1, lineType=8, shift=0)
        
        if tracker.current_tracks[proc-1] is not None:
            
            curr = tracker.current_tracks[proc].astype(np.int32)
            prev = tracker.current_tracks[proc-1].astype(np.int32)
            if abs(curr[0] - prev[0]) < 10:



                dist_to_center = int(math.sqrt((curr[2] - curr[0])**2 + (curr[3] - curr[1])**2) / 2)
                dist_to_circle = int((dist_to_center - radius) / math.sqrt(2))

                new_curr = [curr[0] + dist_to_circle, curr[1] + dist_to_circle,
                           curr[2] - dist_to_circle, curr[3] - dist_to_circle]

                prev_radius = int((prev[2] - prev[0]) / 2)
                dist_to_center = int(math.sqrt((prev[2] - prev[0])**2 + (prev[3] - prev[1])**2) / 2)
                dist_to_circle = int((dist_to_center - prev_radius) / math.sqrt(2))

                new_prev = [prev[0] + dist_to_circle, prev[1] + dist_to_circle,
                           prev[2] - dist_to_circle, prev[3] - dist_to_circle]


                #upper left
                cv2.line(frame, (new_curr[0], new_curr[1]), (new_prev[0], new_prev[1]), (0, 255, 0), thickness=l_thickness)
                #upper right 
                cv2.line(frame, (new_curr[2], new_curr[1]), (new_prev[2], new_prev[1]), (0, 255, 0), thickness=l_thickness)
                #lower left
                cv2.line(frame, (new_curr[0], new_curr[3]), (new_prev[0], new_prev[3]), (0, 255, 0), thickness=l_thickness)
                #lower right
                cv2.line(frame, (new_curr[2], new_curr[3]), (new_prev[2], new_prev[3]), (0, 255, 0), thickness=l_thickness)
            
          
    
    out.write(frame)

    en += 1
    proc += 1
    if proc > 1000:break
    
    
    
out.release()

In [13]:

print(fps)

60.0


In [ ]:
# test multiple videos and show the results using CIRCLE
fps = video.fps
out = cv2.VideoWriter('./demo_vids/output/output_tracker_demo.mp4', cv2.VideoWriter_fourcc(*'MP4V'), fps, (1280,720))

l_thickness = 2
en = 0
proc = 0


all_videos = glob.glob('./demo_vids/*.mp4')

for video_path in all_videos:
    
    video = mmcv.VideoReader(video_path)
    pbar = tqdm(total=video.frame_cnt)
    for frame in video:
        pbar.update(1)

        if all_tracks[proc] is not None:

            result = all_tracks[proc][:4]
            center = (int((result[0] + result[2]) / 2), int((result[1] + result[3]) / 2))
            radius = int((result[2] - result[0]) / 2)
            cv2.circle(frame, center, radius, (0, 255, 0), thickness=1, lineType=8, shift=0)

            if all_tracks[proc-1] is not None:

                curr = all_tracks[proc].astype(np.int32)
                prev = all_tracks[proc-1].astype(np.int32)
                if abs(curr[0] - prev[0]) < 10:



                    dist_to_center = int(math.sqrt((curr[2] - curr[0])**2 + (curr[3] - curr[1])**2) / 2)
                    dist_to_circle = int((dist_to_center - radius) / math.sqrt(2))

                    new_curr = [curr[0] + dist_to_circle, curr[1] + dist_to_circle,
                               curr[2] - dist_to_circle, curr[3] - dist_to_circle]

                    prev_radius = int((prev[2] - prev[0]) / 2)
                    dist_to_center = int(math.sqrt((prev[2] - prev[0])**2 + (prev[3] - prev[1])**2) / 2)
                    dist_to_circle = int((dist_to_center - prev_radius) / math.sqrt(2))

                    new_prev = [prev[0] + dist_to_circle, prev[1] + dist_to_circle,
                               prev[2] - dist_to_circle, prev[3] - dist_to_circle]


                    #upper left
                    cv2.line(frame, (new_curr[0], new_curr[1]), (new_prev[0], new_prev[1]), (0, 255, 0), thickness=l_thickness)
                    #upper right 
                    cv2.line(frame, (new_curr[2], new_curr[1]), (new_prev[2], new_prev[1]), (0, 255, 0), thickness=l_thickness)
                    #lower left
                    cv2.line(frame, (new_curr[0], new_curr[3]), (new_prev[0], new_prev[3]), (0, 255, 0), thickness=l_thickness)
                    #lower right
                    cv2.line(frame, (new_curr[2], new_curr[3]), (new_prev[2], new_prev[3]), (0, 255, 0), thickness=l_thickness)

        out.write(frame) 
        en += 1
        proc += 1
        
    pbar.close()


out.release()

In [ ]:
def visualise_ball_track(all_tracks, video_path, output_path='./demo_vids/output/output_tracker_demo.mp4'):
    
    video = mmcv.VideoReader(video_path)
    fps = video.fps
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'MP4V'), fps, (1280,720))

    l_thickness = 2
    en = 0
    proc = 0
    

    pbar = tqdm(total=video.frame_cnt)
    for frame in video:
        pbar.update(1)

        if all_tracks[proc] is not None:

            result = all_tracks[proc][:4]
            center = (int((result[0] + result[2]) / 2), int((result[1] + result[3]) / 2))
            radius = int((result[2] - result[0]) / 2)
            cv2.circle(frame, center, radius, (0, 255, 0), thickness=1, lineType=8, shift=0)

            if all_tracks[proc-1] is not None:

                curr = all_tracks[proc].astype(np.int32)
                prev = all_tracks[proc-1].astype(np.int32)
                if abs(curr[0] - prev[0]) < 10:



                    dist_to_center = int(math.sqrt((curr[2] - curr[0])**2 + (curr[3] - curr[1])**2) / 2)
                    dist_to_circle = int((dist_to_center - radius) / math.sqrt(2))

                    new_curr = [curr[0] + dist_to_circle, curr[1] + dist_to_circle,
                               curr[2] - dist_to_circle, curr[3] - dist_to_circle]

                    prev_radius = int((prev[2] - prev[0]) / 2)
                    dist_to_center = int(math.sqrt((prev[2] - prev[0])**2 + (prev[3] - prev[1])**2) / 2)
                    dist_to_circle = int((dist_to_center - prev_radius) / math.sqrt(2))

                    new_prev = [prev[0] + dist_to_circle, prev[1] + dist_to_circle,
                               prev[2] - dist_to_circle, prev[3] - dist_to_circle]


                    #upper left
                    cv2.line(frame, (new_curr[0], new_curr[1]), (new_prev[0], new_prev[1]), (0, 255, 0), thickness=l_thickness)
                    #upper right 
                    cv2.line(frame, (new_curr[2], new_curr[1]), (new_prev[2], new_prev[1]), (0, 255, 0), thickness=l_thickness)
                    #lower left
                    cv2.line(frame, (new_curr[0], new_curr[3]), (new_prev[0], new_prev[3]), (0, 255, 0), thickness=l_thickness)
                    #lower right
                    cv2.line(frame, (new_curr[2], new_curr[3]), (new_prev[2], new_prev[3]), (0, 255, 0), thickness=l_thickness)

        out.write(frame) 
        en += 1
        proc += 1

In [6]:
from IPython.display import Video
Video('./demo_vids/output/output_tracker_demo.mp4')